# Notebook for testing showCamera.
Initially written by Michael Reuter.\
Modified by Craig Lage - 24-Sep-21\
Testing with comCam data from 2021-09-23.

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.display as afwDisplay
import lsst.daf.butler as dafButler #Gen3 butler

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
afwDisplay.setDefaultBackend("matplotlib")

In [ ]:
def myCallback(im, ccd, imageSource):
    """Assemble the CCD image.  Just bias subtraction and gain correction"""
    oim = camGeomUtils.rawCallback(im, ccd, imageSource,
                                   subtractBias=False, correctGain=False)
    return oim

In [ ]:
# Instantiate the Gen3 butler
dataPath = "/repo/main"
instrument = "LSSTComCam"
butler = dafButler.Butler(dataPath, 
                          collections=["LSSTComCam/raw/all", "LSSTComCam/calib/unbounded"],
                          instrument=instrument)

In [ ]:
day_obs = 20220113
seq_num = 3

raftName = "R22"
dataId = {"instrument": instrument, "detector.raft": raftName,
          "exposure.day_obs": day_obs, "exposure.seq_num": seq_num}

In [ ]:
# camera has the info necessary to assemble the 9 CCDs
camera = butler.get('camera', instrument=instrument)
# Print the metadata just as a check.
metadata = butler.get('raw.visitInfo', {**dataId, "detector.id": 0})
print(metadata)

In [ ]:
# Print out mean and sigma for each detector
# This is before bias subtraction and gain adjustment.
for det in range(9):
    exp = butler.get('raw', {**dataId, "detector.id": det})
    arr = arr = exp.image.array
    print(det, arr.min(), arr.max(), arr.mean(), arr.std())

In [ ]:
# This is bias subtracted and gain adjusted (see myCallback above), so gives a decent image.
# Note that it is only ~ 200-300 counts after bias subtraction
fig = plt.figure(figsize=(12,12))
disp = afwDisplay.Display(1, "matplotlib")
disp.scale('linear', min = 0.0, max=5.0)

dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, raft=raftName,
                                                       day_obs=day_obs, seq_num=seq_num,
                                                       verbose=True, callback=myCallback,
                                                       background=np.nan),
                              binSize=16, display=disp, overlay=False,
                              title="%d %d %s" % (day_obs, seq_num, dataType))
#plt.savefig("/project/cslage/ComCam/notebooks/alysha/Test.pdf")